In [13]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize

In [2]:
data_demo = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1995-01-01&endtime=1995-01-02&limit=1")
data_demo = data_demo.text
myobject = json.loads(data_demo)
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(myobject,f,ensure_ascii=False,indent=4)

with open('data.json','r') as f:
    data = json.loads(f.read())

usa = pd.json_normalize(data, record_path =['features']) 

usa.drop(0,inplace=True)

#se crea el dataframe "USA" pero solo con las columnas a las que luego se concatenaran los dataframes transformados de las peticiones a la API, 
# mas abajo se explica bien cada seccion del codigo aca presentado

In [3]:
def cantidad_sismos (year,month):   
    conteo = requests.get(f"https://earthquake.usgs.gov/fdsnws/event/1/count?format=geojson&starttime={year}-{month}-01&endtime={year}-{month}-31")#busqueda sobre cantidad de sismos en un determinado mes
    conteo = int(conteo.text.split(",")[0].split(":")[1]) #tomo solo el numero devuelto por el conteo de sismos del mes
    return conteo


In [4]:
def division(year,month):
    sismos = []
    sismo = requests.get(f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={year}-{month}-1&endtime={year}-{month}-5")
    sismos.append(sismo.text)
    sismo = requests.get(f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={year}-{month}-6&endtime={year}-{month}-11")
    sismos.append(sismo.text)
    sismo = requests.get(f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={year}-{month}-12&endtime={year}-{month}-17")
    sismos.append(sismo.text)
    sismo = requests.get(f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={year}-{month}-18&endtime={year}-{month}-23")
    sismos.append(sismo.text)
    sismo = requests.get(f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={year}-{month}-24&endtime={year}-{month}-31")
    #- para aquellos registros mensuales con mas de 20000 sismos, se subdivide en 5 busquedas para evitar tener un error de respuesta por parte de la API
    # las respuestas se suman a la lista de sismos donde pasan al siguiente loop-..
        
    for sismo in sismos:
        myobject = json.loads(sismo) #se cargan los sismos como objetos json

        with open('data.json', 'w', encoding='utf-8') as f:
            json.dump(myobject,f,ensure_ascii=False,indent=4) #se escribe un archivo en formato json para luego poder abrirlos y normalizarlos.

        with open('data.json','r',encoding="utf8") as f:
            data = json.loads(f.read())
            
        new_data = pd.json_normalize(data, record_path =['features']) #se normalizan las listas anidadas dentro del JSON para convertirlo en un dataframe

        global usa #se llama a la variable global usa que contendra todas las cargas de los dataframes
        usa = pd.concat([usa,new_data])   #se concatena el nuevo request ya transformado con el viejo dataframe de los sismos en USA
        f.close() # se cierra el documento json para evitar errores



In [5]:
def data (year,month):
    #aca se ejecuta el mismo codigo que en la funcion "division" solo que es mas corto ya que no hace falta subdividr en etapas de dias el mes, ya que este tiene menos de 20mil registros.
    sismo = requests.get(f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={year}-{month}-01&endtime={year}-{month}-31")
    sismo= sismo.text
    myobject = json.loads(sismo)

    with open('data.json', 'w', encoding='utf-8') as f:
        json.dump(myobject,f,ensure_ascii=False,indent=4)

    with open('data.json','r',encoding="utf8") as f:
        data = json.loads(f.read())

    new_data = pd.json_normalize(data, record_path =['features'])
    
    global usa
    usa = pd.concat([usa,new_data])  
    f.close()



In [ ]:
for year in range(2019,2020):
    for month in range (1,13):        
        if cantidad_sismos(year,month) > 20000:
            division(year,month)                         
            print(year,month)
        else:
            data(year,month)
            print(year,month)

In [16]:
usa.to_csv("usa.csv")

In [23]:
df_usa= pd.read_csv(r'C:\Users\Sol Arena y Mar\Desktop\PF DS\usa.csv')

C:\Users\Sol Arena y Mar\AppData\Local\Temp\ipykernel_8252\3349595259.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_usa= pd.read_csv(r'C:\Users\Sol Arena y Mar\Desktop\PF DS\usa.csv')


In [40]:
df = pd.DataFrame(df_usa)

In [41]:
df

,Unnamed: 0,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,0,Feature,uw10367713,2.30,"34 km W of Neah Bay, Washington",791508669980,1469291003160,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",origin,phase-data,",9.0,0.34620,0.190,301.0,md,earthquake,"M 2.3 - 34 km W of Neah Bay, Washington",Point,"[-125.0893333, 48.3811667, -0.566]"
1,1,Feature,usp0006s4k,4.10,"18 km N of Viña del Mar, Chile",791508586370,1415321832469,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",impact-text,origin,phase-data,",NaN,NaN,0.900,NaN,md,earthquake,"M 4.1 - 18 km N of Viña del Mar, Chile",Point,"[-71.582, -32.863, 7.8]"
2,2,Feature,ci3201757,1.49,"5km NW of Boron, California",791507822240,1455227763893,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",nearby-cities,origin,phase-data,",0.0,0.09880,0.132,90.6,mc,quarry blast,"M 1.5 Quarry Blast - 5km NW of Boron, California",Point,"[-117.686, 35.028, -0.816]"
3,3,Feature,ci3201756,2.78,"19km S of Malibu Beach, California",791507351780,1455217897307,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",nearby-cities,origin,phase-data,",0.0,NaN,0.338,55.9,ml,earthquake,"M 2.8 - 19km S of Malibu Beach, California",Point,"[-118.687, 33.863, 12.298]"
4,4,Feature,ci3201753,1.44,"3km S of Johannesburg, California",791506888320,1455318568986,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",nearby-cities,origin,phase-data,",0.0,0.03310,0.106,146.8,mc,quarry blast,"M 1.4 Quarry Blast - 3km S of Johannesburg, Ca...",Point,"[-117.633, 35.346, -1.051]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3021883,11028,Feature,ak021fe13vhj,1.90,"69 km ENE of Pedro Bay, Alaska",1638317538959,1641423968487,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",origin,phase-data,",NaN,NaN,0.310,NaN,ml,earthquake,"M 1.9 - 69 km ENE of Pedro Bay, Alaska",Point,"[-153.0043, 60.0842, 127.1]"
3021884,11029,Feature,ak021fe12k2v,1.40,"9 km ENE of Fox, Alaska",1638317164125,1641423949173,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",origin,phase-data,",NaN,NaN,0.400,NaN,ml,explosion,"M 1.4 Explosion - 9 km ENE of Fox, Alaska",Point,"[-147.4242, 64.9776, 0]"
3021885,11030,Feature,ci37370060,1.07,"2km WSW of San Bernardino, CA",1638317043980,1638321155271,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",nearby-cities,origin,phase-data,scitech-link,",21.0,0.09794,0.160,104.0,ml,earthquake,"M 1.1 - 2km WSW of San Bernardino, CA",Point,"[-117.3128333, 34.1186667, 13.73]"
3021886,11031,Feature,ci39875087,0.78,"16km S of La Quinta, CA",1638317034660,1638320768737,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,...,",nearby-cities,origin,phase-data,scitech-link,",20.0,0.09303,0.130,111.0,ml,earthquake,"M 0.8 - 16km S of La Quinta, CA",Point,"[-116.3143333, 33.5235, 11.71]"


In [42]:
df=  df.rename(columns=lambda x: x.replace('geometry.', '').replace('properties.', ''))

In [45]:
df.columns

Index(['Unnamed: 0', 'type', 'id', 'mag', 'place', 'time', 'updated', 'tz',
       'url', 'detail', 'felt', 'cdi', 'mmi', 'alert', 'status', 'tsunami',
       'sig', 'net', 'code', 'ids', 'sources', 'types', 'nst', 'dmin', 'rms',
       'gap', 'magType', 'type', 'title', 'type', 'coordinates'],
      dtype='object')